Import Library

In [ ]:
import json
import pandas as pd
import requests
from requests.auth import HTTPBasicAuth
from datetime import date
from datetime import datetime
import os

Define All Functions And Public Variables

In [ ]:
categoryOptionsList = []
today = date.today()
newest_data = []
Start_DateTime = datetime.now()
print("Start At: ",Start_DateTime)
today_date = today.strftime("%Y-%m-%d")
todayDateTime = "LOG/Stock_" + str(datetime.now())
path = '/Users/salehabbas/Developer/Python/pharmacy_stock/' + \
    todayDateTime
try:
    os.mkdir(path)
except OSError as error:
    print(error)


def writefile(FileName, Data):
    json_data = json.dumps(Data, ensure_ascii=False)
    file = open(FileName, "w")
    with open(file.name, "w") as f:
        f.write(json_data)
    file.close()

def category_options():
    category_options_req = requests.get(
        "https://hmis.moh.ps/tr-dev/api/categoryOptions?fields=id,code",
        auth=HTTPBasicAuth('Saleh', 'Test@123')).json()
    store_category(category_options_req["categoryOptions"])
    while category_options_req["pager"]['pageCount'] != category_options_req["pager"]['page']:
        # ? print(categoryOptions_req["pager"]['nextPage'])
        category_options_req = requests.get(
            category_options_req["pager"]['nextPage'], auth=HTTPBasicAuth('Saleh', 'Test@123')).json()
        store_category(category_options_req["categoryOptions"])
    writefile("categoryOptions.json", categoryOptionsList)

def store_category(args):
    for h in args:
        categoryOptionsList.append(h)

print("--------------------------------Mapping And Store attributeCategoryOptions--------------------------------")

def MappingData():
    NewMappingArray = []
    with open('categoryOptions.json') as f1, open(todayDateTime + "/GroupData_" + today_date + ".json") as f2:
        a = json.load(f1)
        b = json.load(f2)
        for c in range(len(b['data'])):
            mCode = ""
            OrgUnit_fromjson = b['data'][c]['orgunit']
            quantity_fromjson = b['data'][c]['q']
            mName = b['data'][c]['m']
            for v in range(len(a)):
                if a[v]['code'] == mName:
                    mCode = a[v]['id']
                    newArray = {"orgunit": OrgUnit_fromjson,
                                "mCode": mCode, "quantity": quantity_fromjson}
                    NewMappingArray.append(newArray)
        if len(b['data']) != len(NewMappingArray):
            category_options()
            MappingData()
        else:
            jsonArrayMapping = json.dumps(NewMappingArray)
            MappingDataArray = json.loads(jsonArrayMapping)
            writefile(todayDateTime + "/GroupData_Mapping_" + today_date +
                        ".json", MappingDataArray)
    return MappingDataArray



if not os.path.exists('categoryOptions.json'):
    category_options()


Get Variables From Loop

In [ ]:
print("--------------------------------Start Loop--------------------------------")

get_org_unit_req = requests.get(
    "https://hmis.moh.ps/tr-dev/api/programs/fnIEoaflGxX?fields=organisationUnits",
    auth=HTTPBasicAuth('Saleh', 'Test@123'))
get_org_unit_data = json.loads(get_org_unit_req.text)
for u in range(len(get_org_unit_data['organisationUnits'])):
    org_unit_id = get_org_unit_data['organisationUnits'][u]['id']
    # print(get_org_unit_data['organisationUnits'][u]['id'])
# !get all TEI For This orgUnit
    get_tei = requests.get(
        "https://hmis.moh.ps/tr-dev/api/trackedEntityInstances?ou=" +
        org_unit_id+"&fields=trackedEntityInstance&lastUpdatedEndDate="
        + today_date + "&lastUpdatedStartDate=" + today_date,
        auth=HTTPBasicAuth('Saleh', 'Test@123'))
    get_tei_data = json.loads(get_tei.text)
    for i in range(len(get_tei_data['trackedEntityInstances'])):
        #loop_array = {"tei": "", "event": "", "orgunit": "", "date": "", "m": "", "q": "", "m1": "", "q1": "", "m2": "",
        #            "q2": "", "m3": "", "q3": "",
        #            "m4": "", "q4": "", "m5": "", "q5": "", "m6": "", "q6": "", "m7": "", "q7": "", "m8": "",
        #            "q8": "", "m9": "", "q9": "", "m10": "", "q10": ""
        #            , "m11": "", "q11": "", "m12": "",
        #            "q12": "", "m13": "", "q13": "",
        #            "m14": "", "q14": "", "m15": "", "q15": "", "m16": "", "q16": "", "m17": "", "q17": "", "m18": "",
        #            "q18": "", "m19": "", "q19": "", "m20": "", "q20": "", "m21": "", "q21": "", "m22": "",
        #            "q22": "", "m23": "", "q23": "", "m24": "", "q24": ""}
        loop_array={}
        tei_id = get_tei_data['trackedEntityInstances'][i]['trackedEntityInstance']
        # print("tei_id= "+tei_id)
        loop_array['tei'] = tei_id
        #! get all event And orgUnit For This TEI
        get_event = requests.get(
            "https://hmis.moh.ps/tr-dev/api/events?trackedEntityInstance=" + tei_id + "&lastUpdatedEndDate=" +
            today_date + "&lastUpdatedStartDate=" + today_date + "&fields=event,orgUnit",
            auth=HTTPBasicAuth('Saleh', 'Test@123'))
        get_event_data = json.loads(get_event.text)

        for j in range(len(get_event_data['events'])):
            event_id = get_event_data['events'][j]['event']
            orgunit = get_event_data['events'][j]['orgUnit']
            loop_array['event'] = event_id
            loop_array['orgunit'] = orgunit
            loop_array['date'] = today_date
            # print("event_id= "+event_id)
            #! get event Data
            get_event_id = requests.get(
                "https://hmis.moh.ps/tr-dev/api/events/" + event_id, auth=HTTPBasicAuth('Saleh', 'Test@123'))
            get_event_id_data = json.loads(get_event_id.text)
            if('dataValues' in json.dumps(get_event_id_data)):
                for k in range(len(get_event_id_data['dataValues'])):
                    loop_array['event'] = event_id
                    event_id_data = get_event_id_data['dataValues'][k]['dataElement']
                    dataelement_date = datetime.strptime(
                        (get_event_id_data['dataValues'][k]['created'])[0:10], '%Y-%m-%d').strftime("%Y-%m-%d")
                    # ? Store All M Values From Event DataElement
                    if event_id_data == "aM2Vn0UUPJB":
                        event_value = get_event_id_data['dataValues'][k]['value']
                        loop_array['m'] = event_value
                    if event_id_data == "WSeukMBwbQ3":
                        event_value = get_event_id_data['dataValues'][k]['value']
                        loop_array['m1'] = event_value
                    if event_id_data == "TORfS27wR0q":
                        event_value = get_event_id_data['dataValues'][k]['value']
                        loop_array['m2'] = event_value
                    if event_id_data == "TnrWDEL4PoR":
                        event_value = get_event_id_data['dataValues'][k]['value']
                        loop_array['m3'] = event_value
                    if event_id_data == "iy166uomfXk":
                        event_value = get_event_id_data['dataValues'][k]['value']
                        loop_array['m4'] = event_value
                    if event_id_data == "ntECq4xEo24":
                        event_value = get_event_id_data['dataValues'][k]['value']
                        loop_array['m5'] = event_value
                    if event_id_data == "Ne2veOUhPw0":
                        event_value = get_event_id_data['dataValues'][k]['value']
                        loop_array['m6'] = event_value
                    if event_id_data == "R2rxr1Z8i4v":
                        event_value = get_event_id_data['dataValues'][k]['value']
                        loop_array['m7'] = event_value
                    if event_id_data == "Dlx79ePwf1g":
                        event_value = get_event_id_data['dataValues'][k]['value']
                        loop_array['m8'] = event_value
                    if event_id_data == "oTCRn8enMzd":
                        event_value = get_event_id_data['dataValues'][k]['value']
                        loop_array['m9'] = event_value
                    if event_id_data == "nTd67mb0PJe":
                        event_value = get_event_id_data['dataValues'][k]['value']
                        loop_array['m10'] = event_value
                    if event_id_data == "g3jYfDWMlju":
                        if get_event_id_data['dataValues'][k]['value']:
                            loop_array['m11'] = "M-301-1001"
                    if event_id_data == "ezwWMPb12eO":
                        if get_event_id_data['dataValues'][k]['value']:
                            loop_array['m12'] = "M-123-1004"
                    if event_id_data == "iifVrszTRRz":
                        if get_event_id_data['dataValues'][k]['value']:
                            loop_array['m13'] = "M-123-1018"
                    if event_id_data == "KIZvcIfQvpT":
                        if get_event_id_data['dataValues'][k]['value']:
                            loop_array['m14'] = "M-192-1010"
                    if event_id_data == "qs0I9NsfUxu":
                        if get_event_id_data['dataValues'][k]['value']:
                            loop_array['m15'] = "M-192-1035"
                    if event_id_data == "YxFGAkRihhj":
                        if get_event_id_data['dataValues'][k]['value']:
                            loop_array['m16'] = "M-123-1030"
                    if event_id_data == "x2qkkLksG0N":
                        if get_event_id_data['dataValues'][k]['value']:
                            loop_array['m17'] = "M-123-1035"
                    if event_id_data == "uuaO52vR7Sc":
                        if get_event_id_data['dataValues'][k]['value']:
                            loop_array['m18'] = "M-192-1016"
                    if event_id_data == "cgKw0UiSSzA":
                        if get_event_id_data['dataValues'][k]['value']:
                            loop_array['m19'] = "M-192-1012"
                    if event_id_data == "A0WmBW0xr3Q":
                        if get_event_id_data['dataValues'][k]['value']:
                            loop_array['m20'] = "M-121-1020"
                    if event_id_data == "igbaI5cMEch":
                        if get_event_id_data['dataValues'][k]['value']:
                            loop_array['m21'] = "M-123-1041"
                    if event_id_data == "tXnR137oYs6":
                        if get_event_id_data['dataValues'][k]['value']:
                            loop_array['m22'] = "M-251-4012"
                    if event_id_data == "joyfXYlQ0aS":
                        if get_event_id_data['dataValues'][k]['value']:
                            loop_array['m23'] = "M-251-4013"
                    if event_id_data == "Oh5NachZvua":
                        if get_event_id_data['dataValues'][k]['value']:
                            loop_array['m24'] = "M-181-1018"

                    
#                     if today_date == dataelement_date:
                    # ? Store All Q Values From Event DataElement
                    if event_id_data == "HS5mppnnRUD":
                        event_value = int(
                            get_event_id_data['dataValues'][k]['value'])
                        loop_array['q'] = event_value
                    if event_id_data == "Kzxa8SKjCdp":
                        event_value = int(
                            get_event_id_data['dataValues'][k]['value'])
                        loop_array['q1'] = event_value
                    if event_id_data == "wmkND48fkvf":
                        event_value = int(
                            get_event_id_data['dataValues'][k]['value'])
                        loop_array['q2'] = event_value
                    if event_id_data == "sm78nae74E0":
                        event_value = int(
                            get_event_id_data['dataValues'][k]['value'])
                        loop_array['q3'] = event_value
                    if event_id_data == "YR7bARfLBay":
                        event_value = int(
                            get_event_id_data['dataValues'][k]['value'])
                        loop_array['q4'] = event_value
                    if event_id_data == "Pl3jCeEVYVG":
                        event_value = int(
                            get_event_id_data['dataValues'][k]['value'])
                        loop_array['q5'] = event_value
                    if event_id_data == "J6s8Ju9Y1xk":
                        event_value = int(
                            get_event_id_data['dataValues'][k]['value'])
                        loop_array['q6'] = event_value
                    if event_id_data == "xFAUNJilvDg":
                        event_value = int(
                            get_event_id_data['dataValues'][k]['value'])
                        loop_array['q7'] = event_value
                    if event_id_data == "tn3XjDR6aUt":
                        event_value = int(
                            get_event_id_data['dataValues'][k]['value'])
                        loop_array['q8'] = event_value
                    if event_id_data == "H2g1tcKI0sK":
                        event_value = int(
                            get_event_id_data['dataValues'][k]['value'])
                        loop_array['q9'] = event_value
                    if event_id_data == "wFeFcxSnOO0":
                        event_value = int(
                            get_event_id_data['dataValues'][k]['value'])
                        loop_array['q10'] = event_value
                    if event_id_data == "gTreHa9FsAJ":
                        event_value = int(
                            get_event_id_data['dataValues'][k]['value'])
                        loop_array['q11'] = event_value
                    if event_id_data == "nubIuNPn6kP":
                        event_value = int(
                            get_event_id_data['dataValues'][k]['value'])
                        loop_array['q12'] = event_value
                    if event_id_data == "UOVMe9Hftr8":
                        event_value = int(
                            get_event_id_data['dataValues'][k]['value'])
                        loop_array['q13'] = event_value
                    if event_id_data == "cMVX1z75Uvh":
                        event_value = int(
                            get_event_id_data['dataValues'][k]['value'])
                        loop_array['q14'] = event_value
                    if event_id_data == "B3rbznpTyjJ":
                        event_value = int(
                            get_event_id_data['dataValues'][k]['value'])
                        loop_array['q15'] = event_value
                    if event_id_data == "nxfbinD79RB":
                        event_value = int(
                            get_event_id_data['dataValues'][k]['value'])
                        loop_array['q16'] = event_value    
                    if event_id_data == "BIP1buozD2e":
                        event_value = int(
                            get_event_id_data['dataValues'][k]['value'])
                        loop_array['q17'] = event_value
                    if event_id_data == "xGxzPWNaSuz":
                        event_value = int(
                            get_event_id_data['dataValues'][k]['value'])
                        loop_array['q18'] = event_value
                    if event_id_data == "Fe9k17OVHPe":
                        event_value = int(
                            get_event_id_data['dataValues'][k]['value'])
                        loop_array['q19'] = event_value     
                    if event_id_data == "wiLMg4LNV3V":
                        event_value = int(
                            get_event_id_data['dataValues'][k]['value'])
                        loop_array['q20'] = event_value     
                    if event_id_data == "z8F0ZEgFeeY":
                        event_value = int(
                            get_event_id_data['dataValues'][k]['value'])
                        loop_array['q21'] = event_value     
                    if event_id_data == "btTycEIdfBr":
                        event_value = int(
                            get_event_id_data['dataValues'][k]['value'])
                        loop_array['q22'] = event_value     
                    if event_id_data == "vO16pLLv3u8":
                        event_value = int(
                            get_event_id_data['dataValues'][k]['value'])
                        loop_array['q23'] = event_value     
                    if event_id_data == "mIKc81UxFte":
                        event_value = int(
                            get_event_id_data['dataValues'][k]['value'])
                        loop_array['q24'] = event_value                                                                                                                                                                                                                                                                 
        newest_data.append(loop_array)
    # print(newest_data)


Store data Togather

In [ ]:
print("--------------------------------Store Data--------------------------------")
if not len(newest_data) == 0:
    jsonStr = json.dumps(newest_data)
    writefile(todayDateTime + "/RowData_" +
                today_date + ".json", json.loads(jsonStr))
    the_big_data_newest_list = []
    for a in range(len(newest_data)):
        def InsertNew(mvalue, qvalue):
            the_big_data_array = {"tei": newest_data[a]['tei'], "event": newest_data[a]['event'],
                                    "orgunit": newest_data[a]['orgunit'], "date": newest_data[a]['date'], "m": mvalue,
                                    "q": qvalue}
            the_big_data_newest_list.append(the_big_data_array)
        if 'm' in newest_data and 'q' in newest_data:
            if newest_data[a]['m'] and newest_data[a]['q']:
                InsertNew(newest_data[a]['m'], newest_data[a]['q'])
        if 'm1' in newest_data and 'q1' in newest_data:
            if newest_data[a]['m1'] and newest_data[a]['q1']:
                InsertNew(newest_data[a]['m1'], newest_data[a]['q1'])
        if 'm2' in newest_data and 'q2' in newest_data:
            if newest_data[a]['m2'] and newest_data[a]['q2']:
                InsertNew(newest_data[a]['m2'], newest_data[a]['q2'])
        if 'm3' in newest_data and 'q3' in newest_data:
            if newest_data[a]['m3'] and newest_data[a]['q3']:
                InsertNew(newest_data[a]['m3'], newest_data[a]['q3'])
        if 'm4' in newest_data and 'q4' in newest_data:
            if newest_data[a]['m4'] and newest_data[a]['q4']:
                InsertNew(newest_data[a]['m4'], newest_data[a]['q4'])
        if 'm5' in newest_data and 'q5' in newest_data:
            if newest_data[a]['m5'] and newest_data[a]['q5']:
                InsertNew(newest_data[a]['m5'], newest_data[a]['q5'])
        if 'm6' in newest_data and 'q6' in newest_data:
            if newest_data[a]['m6'] and newest_data[a]['q6']:
                InsertNew(newest_data[a]['m6'], newest_data[a]['q6'])
        if 'm7' in newest_data and 'q7' in newest_data:
            if newest_data[a]['m7'] and newest_data[a]['q7']:
                InsertNew(newest_data[a]['m7'], newest_data[a]['q7'])
        if 'm8' in newest_data and 'q8' in newest_data:
            if newest_data[a]['m8'] and newest_data[a]['q8']:
                InsertNew(newest_data[a]['m8'], newest_data[a]['q8'])
        if 'm9' in newest_data and 'q9' in newest_data:
            if newest_data[a]['m9'] and newest_data[a]['q9']:
                InsertNew(newest_data[a]['m9'], newest_data[a]['q9'])
        if 'm10' in newest_data and 'q10' in newest_data:
            if newest_data[a]['m10'] and newest_data[a]['q10']:
                InsertNew(newest_data[a]['m10'], newest_data[a]['q10'])
        if 'm11' in newest_data and 'q11' in newest_data:  
            if newest_data[a]['m11'] and newest_data[a]['q11']:
                InsertNew(newest_data[a]['m11'], newest_data[a]['q11'])
        if 'm12' in newest_data and 'q12' in newest_data:  
            if newest_data[a]['m12'] and newest_data[a]['q12']:
                InsertNew(newest_data[a]['m12'], newest_data[a]['q12'])
        if 'm13' in newest_data and 'q13' in newest_data:  
            if newest_data[a]['m13'] and newest_data[a]['q13']:
                InsertNew(newest_data[a]['m13'], newest_data[a]['q13'])
        if 'm14' in newest_data and 'q14' in newest_data:  
            if newest_data[a]['m14'] and newest_data[a]['q14']:
                InsertNew(newest_data[a]['m14'], newest_data[a]['q14'])
        if 'm15' in newest_data and 'q15' in newest_data:  
            if newest_data[a]['m15'] and newest_data[a]['q15']:
                InsertNew(newest_data[a]['m15'], newest_data[a]['q15'])
        if 'm16' in newest_data and 'q16' in newest_data:  
            if newest_data[a]['m16'] and newest_data[a]['q16']:
                InsertNew(newest_data[a]['m16'], newest_data[a]['q16'])
        if 'm17' in newest_data and 'q17' in newest_data:  
            if newest_data[a]['m17'] and newest_data[a]['q17']:
                InsertNew(newest_data[a]['m17'], newest_data[a]['q17'])
        if 'm18' in newest_data and 'q18' in newest_data:  
            if newest_data[a]['m18'] and newest_data[a]['q18']:
                InsertNew(newest_data[a]['m18'], newest_data[a]['q18'])
        if 'm19' in newest_data and 'q19' in newest_data:  
            if newest_data[a]['m19'] and newest_data[a]['q19']:
                InsertNew(newest_data[a]['m19'], newest_data[a]['q19'])
        if 'm20' in newest_data and 'q20' in newest_data:  
            if newest_data[a]['m20'] and newest_data[a]['q20']:
                InsertNew(newest_data[a]['m20'], newest_data[a]['q20'])
        if 'm21' in newest_data and 'q21' in newest_data:  
            if newest_data[a]['m21'] and newest_data[a]['q21']:
                InsertNew(newest_data[a]['m21'], newest_data[a]['q21'])
        if 'm22' in newest_data and 'q22' in newest_data:  
            if newest_data[a]['m22'] and newest_data[a]['q22']:
                InsertNew(newest_data[a]['m22'], newest_data[a]['q22'])
        if 'm23' in newest_data and 'q23' in newest_data:  
            if newest_data[a]['m23'] and newest_data[a]['q23']:
                InsertNew(newest_data[a]['m23'], newest_data[a]['q23'])
        if 'm24' in newest_data and 'q24' in newest_data:  
            if newest_data[a]['m24'] and newest_data[a]['q24']:
                InsertNew(newest_data[a]['m24'], newest_data[a]['q24'])
        
    jsonStr1 = json.dumps(the_big_data_newest_list)
    print(jsonStr1)
else:
    print("EmptyData")
    writefile(todayDateTime + "/JobSummary" +
            today_date + ".json", json.dumps([{"0":"EmptyData"}]))

GroupBy Medication Name And OrgUnit With Sum Quantity
Store As (GroupData.json)

In [ ]:
print("--------------------------------GroupBy--------------------------------")
df = pd.DataFrame(json.loads(jsonStr1))
groupby_values = df.groupby(['m', 'orgunit'])['q'].sum()
js = groupby_values.to_json(orient='table')
writefile(todayDateTime + "/GroupData_" +
            today_date + ".json", json.loads(js))

Store all attributeCategoryOptions Data From API

In [ ]:
print("-------------------------------- Mapping Data --------------------------------")

MappingDataArray = MappingData()
xxxxxxx = json.dumps(MappingDataArray)
salehh = pd.DataFrame(json.loads(xxxxxxx))
fzfzf = salehh.groupby(salehh['orgunit']).sum()
newJsonOrgUnit = fzfzf.to_json(orient='table')
ccoa = json.loads(newJsonOrgUnit)
att_array = []
attribute_json = []
for y in range(len(ccoa['data'])):
    # ? print(ccoa['data'][y]['orgunit'])
    org_in_array = ccoa['data'][y]['orgunit']
    att_req = requests.get(
        "https://hmis.moh.ps/tr-dev/api/events?fields=attributeCategoryOptions,event,lastUpdated&orgUnit=" +
        org_in_array+"&program=fnIEoaflGxX",
        auth=HTTPBasicAuth('Saleh', 'Test@123'))
    att_req_data = json.loads(att_req.text)

    for m in range(len(att_req_data['events'])):
        if att_req_data['events'][m]['attributeCategoryOptions'] not in att_array:
            att_array.append(att_req_data['events']
                                [m]['attributeCategoryOptions'])
            attribute_json.append({"event": att_req_data['events'][m]['event'], "lastUpdated": att_req_data['events']
                                    [m]['lastUpdated'], "attributeCategoryOptions": att_req_data['events'][m]['attributeCategoryOptions']})
        elif att_req_data['events'][m]['attributeCategoryOptions'] in att_array:
            a_list = [a['lastUpdated'] for a in attribute_json if a['attributeCategoryOptions']
                        == att_req_data['events'][m]['attributeCategoryOptions']]
            var_a = a_list[0]
            var_b = att_req_data['events'][m]['lastUpdated']
            if var_b > var_a:
                for a in attribute_json:
                    print(attribute_json)
                    if a['attributeCategoryOptions'] == att_req_data['events'][m]['attributeCategoryOptions']:
                        a['event'] = att_req_data['events'][m]['event']
                        a['lastUpdated'] = att_req_data['events'][m]['lastUpdated']
                    for fgtr in range(len(att_array)):
                        # print(att_array[fgtr])
                        if(att_array[fgtr] == a['attributeCategoryOptions']):
                            att_array[fgtr] = att_req_data['events'][m]['attributeCategoryOptions']
                            # print(att_array[fgtr])

Multiple Loop to check if event exist or not

In [ ]:
    Update_response_array = []
    Create_response_array = []
    print("--------------------------------Update Data Stage--------------------------------")
    for p in range(len(MappingDataArray)):
        mappingArrayCode = MappingDataArray[p]['mCode']
        mappingArrayorgunit = MappingDataArray[p]['orgunit']
        if(mappingArrayCode not in att_array):
            data = {
                "status": "COMPLETED",
                "program": "fnIEoaflGxX",
                "programStage": "fdRCUlObMnG",
                "enrollment": "lzL2rq6vcqw",
                "enrollmentStatus": "ACTIVE",
                "orgUnit": MappingDataArray[p]['orgunit'],
                "eventDate": today_date,
                "dataValues": [
                    {
                        "value": MappingDataArray[p]['quantity'],
                        "dataElement": "LijzB622Z22"
                    }
                ],
                "attributeCategoryOptions": mappingArrayCode
            }
            headers = {'Content-Type': 'application/json'}
            create_event = requests.post("https://hmis.moh.ps/tr-dev/api/events",
                                        data=json.dumps(data), headers=headers, auth=HTTPBasicAuth('Saleh', 'Test@123'))
            att_req_data = json.loads(create_event.text)
            Create_response_array.append(att_req_data)
            print("Create Stock Successfully")
        elif(mappingArrayCode in att_array):
            for t in range(len(attribute_json)):
                if(attribute_json[t]['attributeCategoryOptions'] == mappingArrayCode):
                    storeEventId = attribute_json[t]['event']
            # get every event q value and add this value to it
                    event_req = requests.get(
                        "https://hmis.moh.ps/tr-dev/api/events/"+storeEventId, auth=HTTPBasicAuth('Saleh', 'Test@123'))
                    selected_event_data = json.loads(event_req.text)
                    x_number_on=0
                    for r in range(len(selected_event_data['dataValues'])):
                        if(selected_event_data['dataValues'][r]['dataElement'] == 'LijzB622Z22'):
                            x_number_on=1
                            old_q_value = int(
                                selected_event_data['dataValues'][r]['value'])
                            new_q_value = int(
                                MappingDataArray[p]['quantity'])
                            sum_value = old_q_value+new_q_value
                            selected_event_data['dataValues'][r]['value'] = sum_value
                            data = json.dumps(selected_event_data)
                            headers = {'Content-Type': 'application/json'}
                            Update_req = requests.put(
                                "https://hmis.moh.ps/tr-dev/api/events/"+storeEventId, data=data, headers=headers, auth=HTTPBasicAuth('Saleh', 'Test@123'))
                            Update_response = json.loads(Update_req.text)
                            Update_response_array.append(Update_response)
                            print("Update Stock Successfully")
                    if(x_number_on!=1):
                        just_new={"value":int(MappingDataArray[p]['quantity']),"dataElement":"LijzB622Z22"}
                        selected_event_data['dataValues'].append(just_new)
                        data = json.dumps(selected_event_data)
                        Update_req = requests.put(
                            "https://hmis.moh.ps/tr-dev/api/events/"+storeEventId, data=data, headers=headers, auth=HTTPBasicAuth('Saleh', 'Test@123'))
                        Update_response = json.loads(Update_req.text)
                        Update_response_array.append(Update_response)
                        print("Update Stock Successfully And Create New DataElement")
writefile(todayDateTime+"/Update_ImportSummaries_" +
        today_date+".json", Update_response_array)
writefile(todayDateTime+"/Create_ImportSummaries_" +
        today_date+".json", Create_response_array)

In [ ]:
print(datetime.now()-Start_DateTime)